In [ ]:
import numpy as np
import pandas as pd
import re
import requests as rq
import json

# 處理欄位，轉換成API需要的格式，並下載地號相對應的經緯度

In [ ]:
df1 = pd.read_csv("./combine/presale_4dis.csv")

In [ ]:
# 取出"土地區段位置建物區段門牌"，並轉成List
df2 = df1["土地區段位置建物區段門牌"]
addrList = list(df2)

# 把 "地號" 拿掉
regex01 = "[\u4E00-\u9FFF]+$"
aList1 = []
aList2 = []

for addr in addrList:
    aList1.append(re.split(regex01, addr))
    for a1 in aList1:
        aList2.append(a1[0])


# 取出其中的段號文字
regex02 = "[0-9]+\~[0-9]+"
wordList = []
wList = []

for a2 in aList2:
    wordList.append(re.split(regex02, a2))
    for word in wordList:
        wList.append(word[0])
    

# 取出數字
regex03 = "[\u4E00-\u9FFF]+"
numList = []
nList = []
nList2 = []

for a2 in aList2:
    numList.append(re.split(regex03, a2))
    for num in numList:
        nList.append(num[1])
        for n1 in nList:
            nList2.append(n1.split("~"))

In [ ]:
# 取出"鄉鎮市區"，並轉成List
df3 = df1["鄉鎮市區"]
disList = list(df3)

In [ ]:
# 分割頭尾號碼
hList = []
for h in range(0, len(nList2)):
    hList.append("新北市" + disList[h] + wList[h] + str(nList2[h][0]) + "號")


# 抓出不重複的值
uniList_h = list(pd.unique(hList))


# 加上API要用的 &lands[]=
uniListall_h = []
for i in range(1, len(uniList_h)):
    uniListall_h.append("&lands[]=" + uniList_h[i])


# 因為 API無法一次抓所有經緯度資料，所以分成50筆一組
hList50 = []
hList100 = []
hList120 = []

for i in range(0, 50):
    hList50.append(uniListall_h[i])

for i in range(50, 100):
    hList100.append(uniListall_h[i])
    
for i in range(100, len(uniListall_h)):
    hList120.append(uniListall_h[i])


hstr50 = "".join(hList50)
hstr100 = "".join(hList100)
hstr120 = "".join(hList120)

In [ ]:
# 1-50筆
url01 = "https://twland.ronny.tw/index/search?lands[]=" + uniList_h[0] + hstr50
list01 = rq.get(url01).json()

with open("./data/addr_head01.json", "w") as hf:
    hf.write(json.dumps(list01, ensure_ascii=False))

In [ ]:
# 50-100
url02 = "https://twland.ronny.tw/index/search?lands[]=" + uniList_h[0] + hstr100
list02 = rq.get(url02).json()

with open("./data/addr_head02.json", "w") as hf:
    hf.write(json.dumps(list02, ensure_ascii=False))

In [ ]:
# 100-120
url03 = "https://twland.ronny.tw/index/search?lands[]=" + uniList_h[0] + hstr120
list03 = rq.get(url03).json()

with open("./data/addr_head03.json", "w") as hf:
    hf.write(json.dumps(list03, ensure_ascii=False))

# 整理從API抓到的經緯度資料

In [ ]:
# 抓出json檔中，完整的地號名稱、經度、緯度
AdList01 = []
xList01 = []
yList01 = []

for i in range(0, len(list01["features"])):
    AdList01.append(list01["features"][i]["properties"]["query"]) 
    xList01.append(list01["features"][i]["properties"]["xcenter"])
    yList01.append(list01["features"][i]["properties"]["ycenter"])

In [ ]:
AdList02 = []
xList02 = []
yList02 = []

for i in range(0, len(list02["features"])):
    AdList02.append(list02["features"][i]["properties"]["query"]) 
    xList02.append(list02["features"][i]["properties"]["xcenter"])
    yList02.append(list02["features"][i]["properties"]["ycenter"])

In [ ]:
AdList03 = []
xList03 = []
yList03 = []

for i in range(0, len(list03["features"])):
    AdList03.append(list03["features"][i]["properties"]["query"]) 
    xList03.append(list03["features"][i]["properties"]["xcenter"])
    yList03.append(list03["features"][i]["properties"]["ycenter"])

In [ ]:
# 把分散成3組的資料併起來
AdList_h = AdList01 + AdList02 + AdList03
xList_h = xList01 + xList02 + xList03
yList_h = yList01 + yList02 + yList03

# 轉成 Dataframe格式
addrDic_h = {}
addrDic_h["Addr_h"] = AdList_h
addrDic_h["x_h"] = xList_h
addrDic_h["y_h"] = yList_h

addrDF_h = pd.DataFrame.from_dict(addrDic_h)

In [ ]:
# 存出
addrDF_h.to_csv("./combine/addr_h.csv", index = False)

## 查不到的地號

In [ ]:
misList01 = []
misList02 = []
misList03 = []

for i in range(0, len(list01["notfound"])):
    misList01.append(list01["notfound"][i]["query"])

for i in range(0, len(head02["notfound"])):
    misList02.append(head02["notfound"][i]["query"])

for i in range(0, len(head03["notfound"])):
    misList03.append(head03["notfound"][i]["query"])

# 併起來，因為數量不多，另外用人工方式補齊
misList_h = misList01 + misList02 + misList03

# 把經緯度併回整理好的實價登錄大表

In [ ]:
# 刪除重複值
addrDF_h = addrDF_h.drop_duplicates(keep = "first", inplace = False)

# 把"新北市XX區"拿掉
regex04 = "^[\u4E00-\u9FFF]{6}"
addrDF_h["Addr_h"] = addrDF_h["Addr_h"].str.split(regex04).str.get(1)

# 去掉 "~XX地" 字樣
regex05 = "\~[0-9]+[\u4E00-\u9FFF]"
a = df1["土地區段位置建物區段門牌"].str.split(regex05).str.get(0)
b = df1["土地區段位置建物區段門牌"].str.split(regex05).str.get(1)
df1["地號_h"] = a + b

# 兩張表合併
headDF_1 = pd.merge(df1, addrDF_h, left_on = "地號_h", right_on = "Addr_h", how = "inner")

# 存出報表
headDF_1.to_csv("./combine/presale_location.csv", index = False)